# ## Tinkoff Финтех школа. осень 2018
### Предсказание отклика клиента. Домашняя работа.

Предсказание, что клиент согласится на предложение продукта банка.

** Задание (3 - 8 баллов)**

Предсказать NPV. т.е. задача регрессии и label = 'npv'.

Выбрать лучшие гиперпараметры для каждой из моделей:
    - дерево решений
    - случайный лес
    - бустинг
    
Предсказать NPV на тестовой выборке. 
<br>Поменять DecisionTreeClassifier на DecisionTreeRegressor, аналогично с RandomForest.
<br>В параметрах градиентного бустинга испольовать *'objective': 'binary'*.
<br>Метрику качества использовать **R2** sklearn.metrics.r2_score.

Файл с результатами (test_predict_m.csv) приложить к домашнему заданию.

Все предсказания с `R2 score > 0` хотя бы одной моделью (из 3) получат 3 балла.
<br>Далее ответы студентов упорядочиваются по R2 score.
1. Топ-3 получают 5 дополнительных баллов.
2. Следующие 3 получают 4 дополнительных балла.
3. Следующие 6 получают 3 дополнительных балла.
4. Следующие 10 получают 2 дополнительных балла.

## Решение

In [1]:
import numpy as np
import pandas as pd
import os, sys
import time
from collections import OrderedDict
from tqdm import tqdm
import math
import datetime

import matplotlib.pyplot as plt
%matplotlib inline

from IPython import display
from sklearn.metrics import r2_score

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from collections import OrderedDict
import itertools

In [2]:
dump_path = 'data'
df = pd.read_csv(os.path.join(dump_path,'train_m.csv'))
df_test = pd.read_csv(os.path.join(dump_path,'test_wo_labels_m.csv'))

In [3]:
continous_feats = ['add_base_flg', 's1_last_dates',  's1_prev_dates',  's1_request_flg', 'first_inc_call_dates', 
                   'first_out_call_dates', 's2_last_dates', 's2_prev_dates', 's2_request_flg', 'inc_call_cnt',
                   'last_inc_call_dates', 'last_out_call_dates', 's3_last_dates','s3_prev_dates', 's3_request_flg',
                   'out_call_cnt', 'prev_load_cnt', 'prev_load_dates', 'prev_s2s1', 'prev_s2s2', 'prev_s3s1', 
                   'prev_s3s2', 'prev_s1s1', 's2s1', 's2s2', 'skp_base_flg', 's3s1', 's3s2', 's1s1',
                   'user_contact_base_flg', 's3_available_flg', 's2_available_flg', 's1_available_flg']

categ_feats = ['utm_campaign', 'region', 'mob_provider', 'prev_load_status']

label = 'npv'

### Предобработка

In [4]:
from sklearn.preprocessing import LabelEncoder
le = dict([(f,LabelEncoder()) for f in categ_feats])
for f in categ_feats:
    df[f+'_le'] = le[f].fit_transform(df[f].astype(str))
    
feat_cols = continous_feats + [f + '_le' for f in categ_feats]

In [5]:
df_train = df.query('finish_dttm <  "2018-03-30"')
df_valid = df.query('finish_dttm >= "2018-03-30"')

In [6]:
new_df_train = df_train[feat_cols].fillna(df_train[feat_cols].mean())
new_df_valid = df_valid[feat_cols].fillna(df_train[feat_cols].mean())
new_df_train[label] = df_train[label]
new_df_valid[label] = df_valid[label]

### Модели

#### Решающее дерево

In [7]:
from sklearn.tree import DecisionTreeRegressor

dc = DecisionTreeRegressor(max_depth=10)
dc.fit(new_df_train[feat_cols],new_df_train[label])

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [8]:
train_pred = dc.predict(new_df_train[feat_cols])
valid_pred = dc.predict(new_df_valid[feat_cols])

r2_score_train = r2_score(new_df_train[label],train_pred)
r2_score_valid = r2_score(new_df_valid[label],valid_pred)

print(f'Tree depth {dc.tree_.max_depth}.')
print(f'Decision tree on train sample. R2 {r2_score_train:.3f}.')
print(f'Decision tree on valid sample. R2 {r2_score_valid:.3f}.')

Tree depth 10.
Decision tree on train sample. R2 0.045.
Decision tree on valid sample. R2 -0.117.


In [9]:
parameters_DecisionTree = {
    'max_depth': np.arange(2,20,2),
    'min_samples_leaf': np.exp(np.linspace(2,8,6)).astype(int),
    'max_features': ['sqrt', 'log2', None], 
}

grid_cv_DecisionTree = GridSearchCV(DecisionTreeRegressor(), parameters_DecisionTree, scoring = 'r2', cv = 2)
grid_cv_DecisionTree.fit(df_train[feat_cols].fillna(df_train[feat_cols].mean()),df_train[label])
print(grid_cv_DecisionTree.best_score_)
print('')
print(grid_cv_DecisionTree.best_params_)
print('')
print(grid_cv_DecisionTree.best_estimator_)

-0.01980438200464063

{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2980}

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features='sqrt',
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=2980,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')


In [10]:
train_pred = grid_cv_DecisionTree.best_estimator_.predict(new_df_train[feat_cols])
valid_pred = grid_cv_DecisionTree.best_estimator_.predict(new_df_valid[feat_cols])

r2_score_train = r2_score(new_df_train[label],train_pred)
r2_score_valid = r2_score(new_df_valid[label],valid_pred)

print(f'Tree depth {dc.tree_.max_depth}.')
print(f'Decision tree on train sample. R2 {r2_score_train:.3f}.')
print(f'Dчecision tree on valid sample. R2 {r2_score_valid:.3f}.')

Tree depth 10.
Decision tree on train sample. R2 0.010.
Dчecision tree on valid sample. R2 -0.014.


#### Случайный лес

In [11]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=150, max_depth=5)

rf.fit(new_df_train[feat_cols],new_df_train[label])

train_pred = rf.predict(new_df_train[feat_cols])
valid_pred = rf.predict(new_df_valid[feat_cols])

r2_score_train = r2_score(new_df_train[label],train_pred)
r2_score_valid = r2_score(new_df_valid[label],valid_pred)

print(f'Random forest on train sample. R2 {r2_score_train:.3f}.')
print(f'Random forest on valid sample. R2 {r2_score_valid:.3f}.')

Random forest on train sample. R2 0.016.
Random forest on valid sample. R2 -0.050.


In [12]:
parameters_RandomForest = {
    'n_estimators': [100],
    'max_depth': np.arange(2,15,4),
    'min_samples_leaf': np.exp(np.linspace(3,8,5)).astype(int),
    'max_features': ['sqrt', 'log2', None], 
}

In [13]:
grid_cv_RandomForest = GridSearchCV(RandomForestRegressor(), parameters_RandomForest, scoring = 'r2', cv = 2)
grid_cv_RandomForest.fit(new_df_train[feat_cols],new_df_train[label])
print(grid_cv_RandomForest.best_score_)
print('')
print(grid_cv_RandomForest.best_params_)
print('')
print(grid_cv_RandomForest.best_estimator_)

-0.020070739485257838

{'max_depth': 14, 'max_features': 'sqrt', 'min_samples_leaf': 70, 'n_estimators': 100}

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=14,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=70, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


In [14]:
train_pred = grid_cv_RandomForest.best_estimator_.predict(new_df_train[feat_cols])
valid_pred = grid_cv_RandomForest.best_estimator_.predict(new_df_valid[feat_cols])

r2_score_train = r2_score(new_df_train[label],train_pred)
r2_score_valid = r2_score(new_df_valid[label],valid_pred)

print(f'Decision tree on train sample. R2 {r2_score_train:.3f}.')
print(f'Decision tree on valid sample. R2 {r2_score_valid:.3f}.')

Decision tree on train sample. R2 0.033.
Decision tree on valid sample. R2 -0.005.


#### Градиентный бустинг

In [15]:
lgb_train = lgb.Dataset(new_df_train[feat_cols], df_train[label])
lgb_eval = lgb.Dataset(new_df_valid[feat_cols], df_valid[label], reference=lgb_train)

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'n_estimators': 200
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_eval,
                early_stopping_rounds=10,
                verbose_eval = False
           )

train_pred = gbm.predict(new_df_train[feat_cols], num_iteration=gbm.best_iteration)
valid_pred = gbm.predict(new_df_valid[feat_cols], num_iteration=gbm.best_iteration)

r2_score_train = r2_score(new_df_train[label],train_pred)
r2_score_valid = r2_score(new_df_valid[label],valid_pred)

print(f'gbm on train sample. R2 {r2_score_train:.3f}.')
print(f'gbm on valid sample. R2 {r2_score_valid:.3f}.')

C:\Users\admin\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


gbm on train sample. R2 0.036.
gbm on valid sample. R2 -0.005.


In [16]:
params = {
    'task': ['train'],
    'boosting_type': ['gbdt'],
    'objective': ['regression'],
    'metric': [{'l2'}],
    'num_leaves': [31],
    'learning_rate': [0.05],
    'bagging_fraction': [0.8],
    'bagging_freq': [5],
    'verbose': [1],
    'max_depth': np.arange(2,15,4),
    'min_data_in_leaf': np.exp(np.linspace(3,8,5)).astype(int),
    'feature_fraction': [0.1, 0.3, 0.6, 0.9],
    'n_estimators': [120]
}

In [17]:
def grid_search(model, grid):
    
    lgb_train = lgb.Dataset(new_df_train[feat_cols], new_df_train[label])
    lgb_eval = lgb.Dataset(new_df_valid[feat_cols], new_df_valid[label], reference=lgb_train)
    
    result = {'params':[],'r2_train':[],'r2_valid':[]}
    for param_values in itertools.product(*grid.values()):
        param = dict(zip(grid.keys(),param_values))
        if model == 'gbm':
            gbm = lgb.train(param,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_eval,
                early_stopping_rounds=10,
                verbose_eval = False
           )
            train_pred = gbm.predict(new_df_train[feat_cols], num_iteration=gbm.best_iteration)
            valid_pred = gbm.predict(new_df_valid[feat_cols], num_iteration=gbm.best_iteration)
            r2_train = r2_score(new_df_train[label],train_pred)
            r2_valid = r2_score(new_df_valid[label],valid_pred)
            result['params'].append(param)
            result['r2_train'].append(r2_train)
            result['r2_valid'].append(r2_valid)
    return result

In [18]:
%%time
result = grid_search('gbm', params)

C:\Users\admin\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Wall time: 5min 21s


In [19]:
# Выводим результаты    
pandas_result = (pd.DataFrame(result))

In [20]:
pandas_result.head()

,params,r2_train,r2_valid
0,"{'task': 'train', 'boosting_type': 'gbdt', 'ob...",0.010191,-0.007972
1,"{'task': 'train', 'boosting_type': 'gbdt', 'ob...",0.012757,-0.004726
2,"{'task': 'train', 'boosting_type': 'gbdt', 'ob...",0.012978,-0.002895
3,"{'task': 'train', 'boosting_type': 'gbdt', 'ob...",0.013115,-0.003527
4,"{'task': 'train', 'boosting_type': 'gbdt', 'ob...",0.010133,-0.008198


In [21]:
pandas_result.loc[pandas_result['r2_valid'].idxmax()]

params      {'task': 'train', 'boosting_type': 'gbdt', 'ob...
r2_train                                            0.0205641
r2_valid                                          2.24135e-05
Name: 34, dtype: object

In [22]:
params_of_best = pandas_result.loc[pandas_result['r2_valid'].idxmax()]['params']

In [23]:
params_of_best

{'task': 'train',
 'boosting_type': 'gbdt',
 'objective': 'regression',
 'metric': {'l2'},
 'num_leaves': 31,
 'learning_rate': 0.05,
 'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'verbose': 1,
 'max_depth': 6,
 'min_data_in_leaf': 854,
 'feature_fraction': 0.6,
 'n_estimators': 120}

## Промежуточный вывод
- Как видим, перебор параметров при данной предобработке не дает ничего хорошего
- Далее будет экспериментировать с предобработкой признаков и перебором параметров
- Сосредоточимся на случайном лесе и градиентном бустинге
- В конечный CSV файл добавим прогноз этих двух моделей
- Или одной лучшей

## Предсказание NPV

- Сосредоточимся на градиентном бустинге и random forest
- Поменяем кодирование категориальных признаков (выбрал BaseN)
- Иначе заполним пропуски
- Не будем учитывать все признаки (Смотрим на прошлую часть задания и убираем наихудшие по перемешиванию)
- Добавим новые (Смотрел комбинации категориальных, стало хуже)

In [36]:
dump_path = 'data'
df = pd.read_csv(os.path.join(dump_path,'train_m.csv'))
df_test = pd.read_csv(os.path.join(dump_path,'test_wo_labels_m.csv'))

In [1]:
#df.describe()

In [38]:
continous_feats = ['add_base_flg', 's1_last_dates',  's1_prev_dates',  's1_request_flg', 'first_inc_call_dates', 
                   'first_out_call_dates', 's2_last_dates', 's2_prev_dates', 's2_request_flg', 'inc_call_cnt',
                   'last_inc_call_dates', 'last_out_call_dates', 's3_last_dates','s3_prev_dates', 's3_request_flg',
                   'out_call_cnt', 'prev_load_cnt', 'prev_load_dates', 'prev_s2s1', 'prev_s2s2', 'prev_s3s1', 
                   'prev_s3s2', 'prev_s1s1', 's2s1', 's2s2', 'skp_base_flg', 's3s1', 's3s2', 's1s1',
                   'user_contact_base_flg', 's3_available_flg', 's2_available_flg', 's1_available_flg']

categ_feats = ['utm_campaign', 'region', 'mob_provider', 'prev_load_status']

label = 'npv'

In [39]:
from sklearn.cross_validation import train_test_split
df_train = df.query('finish_dttm <  "2018-03-30"')
df_valid = df.query('finish_dttm >= "2018-03-30"')
#df_train, df_valid = train_test_split(df)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [40]:
len(df_test.columns)

40

In [41]:
len(df_train.columns)

44

In [42]:
important_feats = df_train.columns
st = set(df_train.columns)
for i in df_test.columns:
    for j in df_train.columns:
        if i == j:
            st.remove(i)

In [43]:
st

{'application_flg', 'decision_approve_flg', 'init_cost', 'npv'}

In [44]:
important_feats = set(important_feats)
important_feats.remove('application_flg')
important_feats.remove('decision_approve_flg')
important_feats.remove('init_cost')
important_feats.remove('npv')
important_feats = list(important_feats)

In [45]:
from category_encoders import BaseNEncoder
from category_encoders import OneHotEncoder
enc = BaseNEncoder(cols=categ_feats, base=6)
#enc = OneHotEncoder(cols = categ_feats)
enc.fit(df_train[important_feats])

BaseNEncoder(base=6,
       cols=['utm_campaign', 'region', 'mob_provider', 'prev_load_status'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)

In [46]:
df_train_label = df_train[label]
df_valid_label = df_valid[label]
df_train = df_train[important_feats]
df_valid = df_valid[important_feats]

In [47]:
df_train = enc.transform(df_train)

In [48]:
df_valid = enc.transform(df_valid)

In [49]:
feat_cols = []
for i in df_train.columns:
    if i != 'npv' and i != 'create_dt' and i != 'finish_dttm' and i not in st and i != 's3_last_dates' and i != 'prev_s2s1' and i != 'first_out_call_dates' and i != 'prev_s2s2' and i != 's1_prev_dates':
        feat_cols.append(i)

### Заполнение пропусков

In [50]:
new_df_train = df_train[feat_cols].fillna(df_train[feat_cols].mean())
new_df_valid = df_valid[feat_cols].fillna(df_train[feat_cols].mean())
new_df_train[label] = df_train_label
new_df_valid[label] = df_valid_label

In [51]:
df_train[label] = df_train_label
df_valid[label] = df_valid_label

### LightGBM

In [52]:
lgb_train = lgb.Dataset(new_df_train[feat_cols], new_df_train[label])
lgb_eval = lgb.Dataset(new_df_valid[feat_cols], new_df_valid[label], reference=lgb_train)

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'n_estimators': 115,
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,
                early_stopping_rounds=10,
                verbose_eval = False
           )

train_pred = gbm.predict(new_df_train[feat_cols], num_iteration=gbm.best_iteration)
valid_pred = gbm.predict(new_df_valid[feat_cols], num_iteration=gbm.best_iteration)

r2_score_train = r2_score(new_df_train[label],train_pred)
r2_score_valid = r2_score(new_df_valid[label],valid_pred)

print(r2_score_train)
print(r2_score_valid)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.04380944510424056
0.003251619767259517


In [53]:
def grid_search(model, grid):
    
    lgb_train = lgb.Dataset(new_df_train[feat_cols], new_df_train[label])
    lgb_eval = lgb.Dataset(new_df_valid[feat_cols], new_df_valid[label], reference=lgb_train)
    
    result = {'params':[],'r2_train':[],'r2_valid':[]}
    for param_values in itertools.product(*grid.values()):
        param = dict(zip(grid.keys(),param_values))
        if model == 'gbm':
            gbm = lgb.train(param,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,
                early_stopping_rounds=10,
                verbose_eval = False
           )
            train_pred = gbm.predict(new_df_train[feat_cols], num_iteration=gbm.best_iteration)
            valid_pred = gbm.predict(new_df_valid[feat_cols], num_iteration=gbm.best_iteration)
            r2_train = r2_score(new_df_train[label],train_pred)
            r2_valid = r2_score(new_df_valid[label],valid_pred)
            result['params'].append(param)
            result['r2_train'].append(r2_train)
            result['r2_valid'].append(r2_valid)
    return result

In [54]:
params = {
    'task': ['train'],
    'boosting_type': ['gbdt'],
    'objective': ['regression'],
    'metric': [{'l2'}],
    'num_leaves': [31],
    'learning_rate': [0.05],
    'bagging_fraction': [0.8],
    'bagging_freq': [5],
    'verbose': [1],
    'max_depth': np.arange(2,15,4),
    'min_data_in_leaf': np.exp(np.linspace(3,8,5)).astype(int),
    'feature_fraction': [0.1, 0.3, 0.6, 0.9],
    'n_estimators': [114]
}

In [24]:
%%time
result = grid_search('gbm', params)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


CPU times: user 24min 8s, sys: 1min 2s, total: 25min 11s
Wall time: 8min 10s


In [25]:
# Выводим результаты    
pandas_result = (pd.DataFrame(result))

In [26]:
pandas_result.loc[pandas_result['r2_valid'].idxmax()]

params      {'task': 'train', 'boosting_type': 'gbdt', 'ob...
r2_train                                            0.0222979
r2_valid                                            0.0041399
Name: 34, dtype: object

In [27]:
params_of_best = pandas_result.loc[pandas_result['r2_valid'].idxmax()]['params']

In [28]:
params_of_best

{'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.6,
 'learning_rate': 0.05,
 'max_depth': 6,
 'metric': {'l2'},
 'min_data_in_leaf': 854,
 'num_leaves': 31,
 'objective': 'regression',
 'task': 'train',
 'verbose': 1}

In [58]:
lgb_train = lgb.Dataset(new_df_train[feat_cols], new_df_train[label])
lgb_eval = lgb.Dataset(new_df_valid[feat_cols], new_df_valid[label], reference=lgb_train)

params = {'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.6,
 'learning_rate': 0.05,
 'max_depth': 6,
 'metric': {'l2'},
 'min_data_in_leaf': 854,
 'num_leaves': 31,
 'objective': 'regression',
 'task': 'train',
 'verbose': 1,
'n_estimators':114}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,
                early_stopping_rounds=10,
                verbose_eval = False,
           )

train_pred = gbm.predict(new_df_train[feat_cols], num_iteration=gbm.best_iteration)
valid_pred = gbm.predict(new_df_valid[feat_cols], num_iteration=gbm.best_iteration)

r2_score_train = r2_score(new_df_train[label],train_pred)
r2_score_valid = r2_score(new_df_valid[label],valid_pred)

print(r2_score_train)
print(r2_score_valid)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.02216903725609809
0.003361569411432397


### Это был эксперимент, теперь загружаем данные заново и учим модель на всем df

In [2]:
dump_path = 'data'
df = pd.read_csv(os.path.join(dump_path,'train_m.csv'))
df_test = pd.read_csv(os.path.join(dump_path,'test_wo_labels_m.csv'))

In [3]:
continous_feats = ['add_base_flg', 's1_last_dates',  's1_prev_dates',  's1_request_flg', 'first_inc_call_dates', 
                   'first_out_call_dates', 's2_last_dates', 's2_prev_dates', 's2_request_flg', 'inc_call_cnt',
                   'last_inc_call_dates', 'last_out_call_dates', 's3_last_dates','s3_prev_dates', 's3_request_flg',
                   'out_call_cnt', 'prev_load_cnt', 'prev_load_dates', 'prev_s2s1', 'prev_s2s2', 'prev_s3s1', 
                   'prev_s3s2', 'prev_s1s1', 's2s1', 's2s2', 'skp_base_flg', 's3s1', 's3s2', 's1s1',
                   'user_contact_base_flg', 's3_available_flg', 's2_available_flg', 's1_available_flg']

categ_feats = ['utm_campaign', 'region', 'mob_provider', 'prev_load_status']

label = 'npv'

In [4]:
important_feats = df.columns
st = set(df.columns)
for i in df_test.columns:
    for j in df.columns:
        if i == j:
            st.remove(i)

In [5]:
st

{'application_flg', 'decision_approve_flg', 'init_cost', 'npv'}

In [6]:
important_feats = set(important_feats)
important_feats.remove('application_flg')
important_feats.remove('decision_approve_flg')
important_feats.remove('init_cost')
important_feats.remove('npv')
important_feats = list(important_feats)

In [7]:
from category_encoders import BaseNEncoder
from category_encoders import OneHotEncoder
enc = BaseNEncoder(cols=categ_feats, base=6)
#enc = OneHotEncoder(cols = categ_feats)
enc.fit(df[important_feats])

BaseNEncoder(base=6,
       cols=['utm_campaign', 'region', 'mob_provider', 'prev_load_status'],
       drop_invariant=False, handle_unknown='impute', impute_missing=True,
       return_df=True, verbose=0)

In [8]:
df_label = df[label]
df = df[important_feats]

In [9]:
df = enc.transform(df)

In [10]:
feat_cols = []
for i in df.columns:
    if i != 'npv' and i != 'create_dt' and i != 'finish_dttm' and i not in st and i != 's3_last_dates' and i != 'prev_s2s1' and i != 'first_out_call_dates' and i != 'prev_s2s2' and i != 's1_prev_dates':
        feat_cols.append(i)

In [11]:
new_df = df[feat_cols].fillna(df[feat_cols].mean())
new_df[label] = df_label

In [12]:
lgb_train = lgb.Dataset(new_df[feat_cols], new_df[label])

params = {'bagging_fraction': 0.8,
 'bagging_freq': 5,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.6,
 'learning_rate': 0.05,
 'max_depth': 6,
 'metric': {'l2'},
 'min_data_in_leaf': 854,
 'num_leaves': 31,
 'objective': 'regression',
 'task': 'train',
 'verbose': 1,
'n_estimators':114}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,
                early_stopping_rounds=10,
                verbose_eval = False,
           )

train_pred = gbm.predict(new_df[feat_cols], num_iteration=gbm.best_iteration)

r2_score_train = r2_score(new_df[label],train_pred)

print(r2_score_train)

/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.023858856943278606


### Предобработка и предсказание для теста

In [13]:
df_test = enc.transform(df_test)

In [14]:
new_df_test = df_test[feat_cols].fillna(df[feat_cols].mean())

In [15]:
test_pred = gbm.predict(new_df_test[feat_cols], num_iteration=gbm.best_iteration)

In [2]:
#new_df_test.head()

test_predict_m.csv

In [17]:
test_predict_m = pd.DataFrame(test_pred, columns=['npv'])

In [29]:
#df_test['decision_tree_predict'] = <your code here>
#df_test['random_forest_predict'] = <your code here>
df_test['lgbm_predict'] = test_predict_m

df_test['student_name'] = 'a.polyakov'  # Поменяйте на свои имя фамилию

In [30]:
df_test[['hl_rk','lgbm_predict','student_name']].to_csv(
    os.path.join(dump_path,'test_predict_m.csv'),index=False)

In [31]:
a = pd.read_csv('data/test_predict_m.csv')

In [3]:
#a